In [17]:
import boto3
from datetime import datetime

startTimeObj = datetime.now()

session = boto3.session.Session()
s3_resource = boto3.resource("s3")
ec2_resource = boto3.resource("ec2")
ec2_client = boto3.client('ec2')

In [18]:
# %load AWS_functions.py
import uuid
import subprocess    
import os
from dotenv import load_dotenv
import paramiko
load_dotenv(dotenv_path="../.env")

def aws_create_bucket_name(bucket_name):
    """Creta a unique bucket name
    Keyword arguments:
    bucket_name: string
    return name + uuid string
    """
    return '-'.join([bucket_name, str(uuid.uuid4())])

def aws_create_bucket(bucket_name, s3_connection, current_region):
    """Create an AWS Bucket
    Keyword arguments:
    bucket name: string
    s3_connection: boto3 S3 Resource
    return bucket_name, bucket_response
    """
    # Create Configurations
    configs = {}
    if current_region != "us-east-1":
        config["LocationConstraint"] = current_region

    bucket_name = aws_create_bucket_name(bucket_name)
    
    if current_region == "us-east-1":
        bucket_response = s3_connection.create_bucket(
            Bucket=bucket_name)
    else:
        bucket_response = s3_connection.create_bucket(
            Bucket=bucket_name,
            CreateBucketConfiguration={"LocationConstraint":current_region})
        
    print(bucket_name, current_region)
    return bucket_name, bucket_response

def aws_upload_files_to_bucket(path_to_src, bucket_name):
    """Use AWS CLI to sync a local folder to AWS Bucket"""
    path_to_s3 = "s3://" + bucket_name

    try:
        result = os.popen('aws s3 sync '+path_to_src+' '+path_to_s3+' --exclude "model/*" --acl private').read()
        print(result)
        return True
    except:
        print("upload failed")
        return False

def aws_list_objects(bucket_name, s3_resource):
    bucket=s3_resource.Bucket(bucket_name)
    res = []
    for obj_version in bucket.object_versions.all():
        res.append({
            'file_name': obj_version.object_key,
            'VersionId': obj_version.id
        })
    return res, bucket

def aws_delete_all_objects(bucket_name, s3_resource):
    """Delete all objects inside a bucket"""
    resources, bucket = aws_list_objects(bucket_name, s3_resource)
    bucket.delete_objects(Delete={'Objects': resources})
    return bucket

def aws_delete_bucket(bucket_name, s3_resource):
    """delete bucket and all its content"""
    bucket = aws_delete_all_objects(bucket_name, s3_resource)
    bucket.delete()

def aws_list_buckets(bucket_name, s3_resource):
    return s3_resource.buckets.all()

def aws_download_objects(path_to_src, bucket_name):
    """Use AWS CLI to sync a local folder to AWS Bucket"""
    path_to_s3 = "s3://" + bucket_name

    try:
        result = os.popen('aws s3 sync '+path_to_s3+' '+path_to_src).read()
        print(result)
        return True
    except:
        print("upload failed")
        return False  

def aws_create_ec2_name(bucket_prefix):
    """Create a unique ec2 name"""
    return '-'.join([bucket_prefix, str(uuid.uuid4())])

def aws_user_data_script(bucket_name):
    user_data = [
        "#cloud-boothook",
        "#!/bin/bash",
        "yum update -q -y",
        "yum install automake fuse fuse-devel gcc-c++ git libcurl-devel libxml2-devel make openssl-devel -q -y",
        "git clone https://github.com/s3fs-fuse/s3fs-fuse.git /tmp/s3fs-fuse",
        "cd /tmp/s3fs-fuse",
        "./autogen.sh",
        "./configure --prefix=/usr --with-openssl",
        "make",
        "make install",
        'echo "' + os.getenv("AWS_ACCESS_ID") + ':' + os.getenv("AWS_SECRET_ACCESS_KEY") + '" > /tmp/passwd-s3fs',
        "mv -f /tmp/passwd-s3fs /etc",
        "chmod 640 /etc/passwd-s3fs",
        "mkdir -p /mys3bucket",
        "chown ec2-user:ec2-user /mys3bucket",
        "s3fs " + bucket_name + " -o use_cache=/tmp -o uid=500 -o mp_umask=002 -o multireq_max=5 -o allow_other /mys3bucket",
        # "while read requirement; do conda install --yes $requirement; done < /mys3bucket/requirements.txt"
    ]
    return "\n".join(user_data)

def aws_create_ec2_instance(ec2_name, security_group, user_data, ec2_resource, image_id = "ami-062c42cbecc1d5ec0", instance_type="t2.medium"):
    pem_key =  os.getenv("AWS_PEM_KEY")
    ec2_name = aws_create_ec2_name(ec2_name)

    instances = ec2_resource.create_instances(
        TagSpecifications=[
            {
                'ResourceType': 'instance',
                'Tags': [
                    {
                        'Key': 'Name',
                        'Value': ec2_name
                    },
                ]
            }
        ],
        ImageId=image_id,
        MinCount=1,
        MaxCount=1,
        InstanceType=instance_type,
    #     InstanceType='t2.medium',
        KeyName=pem_key,
        SecurityGroupIds=security_group, #Bring your own!
        UserData=user_data
    )

    return instances[0]

def aws_stop_ec2(instance_id, ec2_client):
    return ec2_client.stop_instances(
        InstanceIds=[
            instance_id,
        ]
    )

def exec_ssh_cmd(public_dns_name = "", commands = ""):
    pem_key = os.getenv("AWS_PEM_KEY")
    pem_location = os.getenv("AWS_PEM_LOCATION")
    ssh = paramiko.SSHClient()
    ssh.set_missing_host_key_policy(paramiko.AutoAddPolicy())
    privkey = paramiko.RSAKey.from_private_key_file(pem_location+pem_key+'.pem')
    ssh.connect(public_dns_name,username="ec2-user",pkey=privkey)
    
    result = []
    for cmd in commands:
        stdin, stdout, stderr = ssh.exec_command(cmd)
        stdin.flush()
        data = stdout.read().splitlines()
        data = [l.decode("utf-8") for l in data]

        result.append({"cmd":cmd,"response":data})

    ssh.close()
    return result

<a id="top"></a>
# Following Workflow

### [File Storage](#File_Storage)
* [Create](#File_Storage_Create) an **S3 Bucket**
* [Load](#File_Storage_Load) csv file and script to execute
* [Delete](#File_Storage_Delete) object and content
* [List](#File_Storage_List) objects

### [Execute Code](#Execute_Code)
#### [EC2](#Execute_Code_EC2) Miniconda Image
* [Create](#Execute_Code_EC2_Create) the EC2 Instance
* [Mount](#Execute_Code_EC2_Mount) S3 Storage
* [Execute](#Execute_Code_EC2_Script) Script
* [Stop](#Execute_Code_EC2_Stop) \ Terminate server

### [Deploy our Model](#Deploy-Model)

### Requirements
pycloud environment
```
conda env create -f pycloud.env.ymp --force
```

From Command line execute `aws configure` in order to setup your AWS Access Key and AWS Secret Key

Local Test Code is located under `example_src`

[Go back to top](#top)
## File Storage<a id="File_Storage"></a>
Code to create an S3 Storage
https://realpython.com/python-boto3-aws-s3/#creating-a-bucket

### Create S3 Object<a id="File_Storage_Create"></a>

In [19]:
# %run AWS_functions.py

# bucket_name, first_response = aws_create_bucket(
#     bucket_name='iris-train', 
#     s3_connection=s3_resource.meta.client,
#     current_region=session.region_name)

### Load Files into S3 Object<a id="File_Storage_Load"></a>
There is no native folder sync within Python SDK, so using aws command to solve for this problem!

In [20]:
# %run AWS_functions.py

# success = aws_upload_files_to_bucket(
#     path_to_src="../example_src/iris", 
#     bucket_name=bucket_name)

# print("Uploaded was a", success)

[Go to EC2](#Execute_Code_EC2)
[Go to SageMaker](#Execute_Code_SM)

### Delete S3 Object and Content<a id="File_Storage_Delete"></a>
Remove all resources and delete bucket!<br>
**This does not back-up anything!**

In [21]:
# %run AWS_functions.py

# aws_delete_bucket(bucket_name, s3_resource)    

### List S3 Buckets<a id=File_Storage_List></a>
Quickly list and clean up all buckets created by with iris-trian in name

In [22]:
import boto3
s3_resource = boto3.resource('s3')
for bucket in s3_resource.buckets.all():
  if "iris-train" in bucket.name:
    print(bucket.name)
#     aws_delete_bucket(bucket.name, s3_resource)

iris-train-12746b8f-be9f-4ef8-8c06-38323c33c122
iris-train-e41a03ef-6946-44a0-b326-b6784f36a9f3


In [23]:
bucket_name = "iris-train-12746b8f-be9f-4ef8-8c06-38323c33c122"

[Go back to top](#top)
## Execute Code<a id=Execute_Code></a>

## EC2 Instance<a id=Execute_Code_EC2></a>

### Create EC2 Instance<a id=Execute_Code_EC2_Create></a>
https://blog.ipswitch.com/how-to-create-an-ec2-instance-with-python

You need to bring your own:
* Security Group
* pem key

We will be building the following EC2
* MiniConda - ami-062c42cbecc1d5ec0
* t2.medium

I built my own security group and granted ssh access.

Use a bash script to create the S3 Mount, go [here](#Execute_Code_EC2_Mount) to see details

### Mount S3 onto EC2 Instance<a id=Execute_Code_EC2_Mount></a>
https://cloudkul.com/blog/mounting-s3-bucket-linux-ec2-instance/

Using existing EC2 in AWS
Need to leverage API to create EC2 and mount determining setup

**Required setup on EC2**
```
sudo yum update
sudo yum install automake fuse fuse-devel gcc-c++ git libcurl-devel libxml2-devel make openssl-devel
git clone https://github.com/s3fs-fuse/s3fs-fuse.git
cd s3fs-fuse
./autogen.sh
./configure --prefix=/usr --with-openssl
make
sudo make install
```

You must create an IAM role for S3 Mounting, for sake of simplicity, i'm using my Admin IAM Access
```
sudo touch /etc/passwd-s3fs
sudo vim /etc/passwd-s3fs
```
Provide `Your_accesskey:Your_secretkey` inside the file
```
sudo chmod 640 /etc/passwd-s3fs
```

Let's mount it!, replace iris-trainc0e3588c-d9bb-4699-821c-1883670ace42 with your bucket name
uid=500 is ec2-user account
```
sudo mkdir /mys3bucket
sudo chown ec2-user:ec2-user /mys3bucket
s3fs iris-trainc0e3588c-d9bb-4699-821c-1883670ace42 -o use_cache=/tmp -o allow_other -o uid=500 -o mp_umask=002 -o multireq_max=5 /mys3bucket
```

Validate
```
df -Th
```

Mount at reboot
```
vi /etc/rc.local
/usr/bin/s3fs iris-trainc0e3588c-d9bb-4699-821c-1883670ace42 -o use_cache=/tmp -o allow_other -o uid=500 -o mp_umask=002 -o multireq_max=5 /mys3bucket
```
**or** add it to the User Data at execution!

In [24]:
%run AWS_functions.py

from IPython.display import display
import socket
import time
from time import sleep


# create a new EC2 instance
user_data = aws_user_data_script(bucket_name)

instance = aws_create_ec2_instance(
    ec2_name="iris-train", 
    security_group=['sg-0d24aec64507df8b5'], 
    user_data=user_data, 
    ec2_resource=ec2_resource,
    image_id = "ami-062c42cbecc1d5ec0", 
    instance_type="t2.medium")

print("instance id: ",instance.id)

instance id:  i-0a24097ed20c3ee63


In [25]:
#Provide status when instance is finally up!
retries = 10
retry_delay = 10
retry_count = 0

print("Wait till instance state changes to running")
instance.wait_until_running()
instance = ec2_resource.Instance(id=instance.id)
print("Instance State Up, waiting for boot-up")

waiting_status = "instance is still loading retrying . . . "
dh = display(waiting_status,display_id=True)

while retry_count <= retries:
    sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    result = sock.connect_ex((instance.public_ip_address,22))
    if result == 0:
        print("Instance is UP & accessible on port 22, the IP address is:  ",instance.public_ip_address)
        break
    else:
        if len(waiting_status) < 50:
            waiting_status += ". "
        else:
            waiting_status = waiting_status[0:41]

        dh.update(waiting_status)
        time.sleep(retry_delay)

Wait till instance state changes to running
Instance State Up, waiting for boot-up


'instance is still loading retrying . . . . '

Instance is UP & accessible on port 22, the IP address is:   54.159.6.87


Run the following commands via SSH

From what I can tell miniconda app happens after UserData is complete, thus no installing conda environment

We will execute the following command through ssh client via root access
```
sudo su
while read requirement; do conda install --yes $requirement; done < /mys3bucket/requirements.txt
```

In [26]:
%run AWS_functions.py

commands = [
    "mkdir /mys3bucket/output",
    "mkdir /mys3bucket/model",
    "echo 'for requirement in `cat /mys3bucket/requirements.txt` ; do  conda install --yes  ${requirement}  ;  done' > /tmp/setup.sh",
    "chmod +x /tmp/setup.sh",
    "sudo  -i /tmp/setup.sh",
    "conda list"
]

ssh_result = exec_ssh_cmd(instance.public_ip_address,commands)

#Print the last line conda list
[print(l) for l in ssh_result[5]["response"]]

# packages in environment at /opt/conda:
#
# Name                    Version                   Build  Channel
_libgcc_mutex             0.1                        main  
asn1crypto                0.24.0                   py37_0  
blas                      1.0                         mkl  
ca-certificates           2019.10.16                    0  
certifi                   2019.9.11                py37_0  
cffi                      1.11.5           py37he75722e_1  
chardet                   3.0.4                    py37_1  
conda                     4.5.12                   py37_0  
conda-env                 2.6.0                         1  
cryptography              2.4.2            py37h1ba5d50_0  
idna                      2.8                      py37_0  
intel-openmp              2019.4                      243  
joblib                    0.14.0                     py_0  
libedit                   3.1.20181209         hc058e9b_0  
libffi                    3.2.1                hd8

[None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None,
 None]

### Execute Python Script<a id=Execute_Code_EC2_Script></a>

Execute the train.py file!
```
cd /mys3bucket
python train.py
```

In [27]:
%run AWS_functions.py

ssh_result = exec_ssh_cmd(
    public_dns_name = instance.public_ip_address,
    commands = ["cd /mys3bucket/; python iris_train.py"])

for val in ssh_result[0]["response"]:
    print(val) 

load data
trian model
save model


### Wait for S3 to refresh then download and kill instance

In [28]:
%run AWS_functions.py
import time
from time import sleep

while True:
    resources, bucket = aws_list_objects(
        bucket_name=bucket_name,
        s3_resource=s3_resource)
    models = [ res["file_name"] for res in resources if "model/iris" in res["file_name"]]
    if len(models) > 0:
        aws_download_objects(
            path_to_src="../example_src/iris",
            bucket_name=bucket_name
        )
        
        response = aws_stop_ec2(
        instance_id = instance.id, 
        ec2_client = ec2_client)

        print( response )
        
        break
    time.sleep(5)
    

Completed 2.2 KiB/~2.2 KiB (6.1 KiB/s) with ~1 file(s) remaining (calculating...)
download: s3://iris-train-12746b8f-be9f-4ef8-8c06-38323c33c122/iris_train.py to ../example_src/iris/iris_train.py
Completed 2.2 KiB/~2.2 KiB (6.1 KiB/s) with ~0 file(s) remaining (calculating...)
Completed 10.8 KiB/10.8 KiB (7.9 KiB/s) with 1 file(s) remaining                 
download: s3://iris-train-12746b8f-be9f-4ef8-8c06-38323c33c122/model/iris-rf.pkl to ../example_src/iris/model/iris-rf.pkl

{'StoppingInstances': [{'CurrentState': {'Code': 64, 'Name': 'stopping'}, 'InstanceId': 'i-0a24097ed20c3ee63', 'PreviousState': {'Code': 16, 'Name': 'running'}}], 'ResponseMetadata': {'RequestId': '9da6c09b-5f3b-4d8d-88ad-2bece8bf9624', 'HTTPStatusCode': 200, 'HTTPHeaders': {'content-type': 'text/xml;charset=UTF-8', 'content-length': '579', 'date': 'Tue, 26 Nov 2019 02:08:12 GMT', 'server': 'AmazonEC2'}, 'RetryAttempts': 0}}


In [29]:
endTimeObj = datetime.now()

print("Total Elapse Time", endTimeObj - startTimeObj)

Total Elapse Time 0:06:05.705061


# Deploy Model

https://medium.com/@patrickmichelberger/how-to-deploy-a-serverless-machine-learning-microservice-with-aws-lambda-aws-api-gateway-and-d5b8cbead846

We used a mircoserver flask and zappa to automate lambda, s3, and api gateway

*Assumptions*
* Flask app created in the project folder under api folder and works locally
* Pickle \ joblib file uploaded to s3
* virtualenv installed `pip install virtualenv`

1. Setup a lambda virtualenv from your project folder
```
cd example_src\iris
virtualenv lambda
source lambda/bin/activate
pip install flask zappa sklearn numpy scipy
```
2. Test your flask locally by running `python api/app.py`
3. Initialize zappa by `zappa init`
    * Environment: dev
    * S3 Bucket: use default
    * App Function: use default `api.app.app`
    * Globally: n
*Should look like this*
```
{
    "dev": {
        "app_function": "api.app.app",
        "aws_region": "us-east-1",
        "profile_name": "default",
        "project_name": "iris",
        "runtime": "python3.7",
        "s3_bucket": "zappa-telpd5in0"
    }
}
```
4) Deploy zappa 
```zappa deploy dev```<br>
5) Test Model<br>
```
curl -d '{"data":[[4, 2.1,1,0.4], [6.2, 1,3,2]]}' -X POST https://sgj5uofirg.execute-api.us-east-1.amazonaws.com/dev
```